<a href="https://colab.research.google.com/github/datasistah/ml_sytem_design_course/blob/main/FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import requests
import os

data_url = "https://raw.githubusercontent.com/jamescalam/data/main/sentence_embeddings_15K/"
s=[]
# create data directory to store data
if not os.path.exists('./data'):
    os.mkdir('./data')

# download the numpy binary files (dense vectors)
for i in range(57):
    if i < 10:
        i = '0' + str(i)
    res = requests.get(data_url+f"embeddings_{i}.npy")

    with open(f'./data/embeddings_{i}.npy', 'wb') as fp:
        for chunk in res:
            fp.write(chunk)
           
    print('.', end='')

# and download the respective text file
res = requests.get(f"{data_url}sentences.txt")
with open(f"./data/sentences.txt", 'wb') as fp:
    for chunk in res:
        fp.write(chunk)

.........................................................

In [2]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [3]:
import pandas as pd

import requests
from io import StringIO
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [4]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [5]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

4802

In [6]:


# each of these dataset have the same structure, so we loop through each creating our sentences data
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    
    smalldata = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
    # data=data.append(smalldata,ignore_index=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(smalldata[1].tolist())
    sentences.extend(smalldata[2].tolist())

<ipython-input-6-4e63cc361411>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  smalldata = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 191: expected 3 fields, saw 4
Skipping line 206: expected 3 fields, saw 4
Skipping line 295: expected 3 fields, saw 4
Skipping line 695: expected 3 fields, saw 4
Skipping line 699: expected 3 fields, saw 4

<ipython-input-6-4e63cc361411>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  smalldata = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 104: expected 3 fields, saw 4
Skipping line 181: expected 3 fields, saw 4
Skipping line 317: expected 3 fields, saw 4
Skipping line 412: expected 3 fields, saw 5
Skipping line 508: expected 3 fields, saw 4

<ipython-input-6-4e63cc36141

In [7]:
# smalldata.head()

In [8]:
# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [9]:
len(sentences)

14504

In [10]:
import numpy as np
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.4 MB/s eta 0:00:00


convert raw text(sentences)> into embeddings [using an encoder] sentence-transformers > save it in FAISS >raw text query > into an Embedding > Eucliden distance to retrieve the text closest to the query (or the best one for similarity)

(checkout Faiss versus Pinecone...more commercialized)

In [12]:
!pip install -U sentence-transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
len(sentences)

14504

In [13]:
# !git clone https://github.com/jamescalam/data.git data-embeddings

In [14]:
# path = '/content/data-embeddings/sentence_embeddings_15K/'
# sentence_embeddings = []
# for i in range(0,57):
#     # if i < 10:
#     #     i = '0' + str(i)
#     res = path+f"embeddings_{i}.npy"
#     print(res)
#     sm = np.load(res)
#     sentence_embeddings.append(sm)

In [15]:
# arr = np.concatenate(sentence_embeddings)

In [16]:
# arr.shape

In [17]:
# sentences = sentences[:100]

In [18]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [21]:
%%time
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

CPU times: user 17.1 s, sys: 1.06 s, total: 18.2 s
Wall time: 23.1 s


(14504, 768)

In [24]:
sentences[0]

'The brown dog is running on the grass.'

In [22]:
sentence_embeddings[0]

array([-1.18922114e-01,  2.21482024e-01, -7.49487340e-01, -1.82283878e-01,
        1.05901986e-01,  8.29388872e-02, -6.89999819e-01, -8.38628173e-01,
       -7.20678508e-01, -2.37614378e-01, -4.77328330e-01,  1.76046804e-01,
        2.27121159e-01, -2.12780148e-01,  1.41274893e+00, -3.50996494e-01,
       -6.33713961e-01, -6.38390243e-01, -5.80772638e-01,  4.82596964e-01,
       -1.24125409e+00, -5.55478275e-01, -1.83758754e-02, -4.08920735e-01,
       -3.92821705e-04, -5.99457026e-02,  7.77442157e-01,  1.24351931e+00,
       -1.95218539e+00,  1.73254922e-01, -2.73037910e-01, -4.74173546e-01,
        8.48307192e-01,  2.61081047e-02,  3.36865872e-01,  3.48784894e-01,
       -8.24795663e-01, -3.64450097e-01,  1.86404511e-01, -1.99833059e+00,
        4.32789236e-01,  1.28578377e+00, -1.16913152e+00,  2.71737397e-01,
       -8.17041576e-01, -1.74693216e-03, -8.09913814e-01,  3.22661281e-01,
       -6.48633003e-01,  2.00438410e-01,  2.49205306e-01,  9.17910755e-01,
       -1.98672628e+00, -

In [25]:
# with open(f'./sim_sentences/embeddings_X.npy', 'wb') as fp:
#     np.save(fp, sentence_embeddings[0:256])

In [26]:
# # saving data
# split = 256
# file_count = 0
# for i in range(0, sentence_embeddings.shape[0], split):
#     end = i + split
#     if end > sentence_embeddings.shape[0] + 1:
#         end = sentence_embeddings.shape[0] + 1
#     file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
#     with open(f'./sim_sentences/embeddings_{file_count}.npy', 'wb') as fp:
#         np.save(fp, sentence_embeddings[i:end, :])
#     print(f"embeddings_{file_count}.npy | {i} -> {end}")
#     file_count = int(file_count) + 1

In [27]:
import faiss
# sentence_embeddings = arr

In [28]:
d = sentence_embeddings.shape[1]
d

768

In [29]:
sentences[0]

'The brown dog is running on the grass.'

In [30]:
sentence_embeddings[0].shape

(768,)

In [31]:
index = faiss.IndexFlatL2(d) #euclidean method


In [32]:
index.is_trained

True

In [33]:
index.add(sentence_embeddings)

In [34]:
index.ntotal

14504

converted raw text to embeddings > saved the embeddings into FAIS and we are going to use Euclidean distance to measure the distances

In [35]:
sentences[:5]

['The brown dog is running on the grass.',
 'A person is dancing in the rain',
 'A man wearing a white helmet is in the water.',
 'There is no woman slicing a green pepper',
 "A bubble blew, dyeing the girl's shirt red."]

In [37]:
k = 4 #I want the top 4 results from the query
xq = model.encode(["Dog is running in the bushes"]) #convert query into embedding

#check by putting question in chatgpt to get a reworded sentence and it should return the same match

Converted the query into an embedding and now we will run Euclidean search on the entire corpus and compare it to the query

In [38]:

%%time
D, I = index.search(xq, k)  # search
print(I)

[[4751 3846 2220 2649]]
CPU times: user 19.2 ms, sys: 17 µs, total: 19.3 ms
Wall time: 22.9 ms


In [39]:
[f'{i}: {sentences[i]}' for i in I[0]]

['4751: Two dogs run through the brush.',
 '3846: Two dogs are running on the grass in front of the trees.',
 '2220: two dogs running in grass',
 '2649: A dog is running along the grass.']


Search engine
-Trying to search through every document takes longer, but to make the smaller search space


Quantization
-The sample space becomes smaller when you apply quantization. Breaks ups sample space into chunks. Speeding up the process.  It only compares it to the closest chunk 

In [41]:
nlist = 100
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist) #applying quantizer, increases speed

In [42]:
index.is_trained

False

In [43]:
index.train(sentence_embeddings)
index.is_trained

True

In [44]:
index.add(sentence_embeddings)
index.ntotal

14504

In [45]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[4751 3846 2220 2649]]
CPU times: user 1.98 ms, sys: 0 ns, total: 1.98 ms
Wall time: 1.99 ms


In [46]:
index.nprobe = 10

In [47]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[4751 3846 2220 2649]]
CPU times: user 3.97 ms, sys: 0 ns, total: 3.97 ms
Wall time: 3.9 ms


In [48]:
[f'{i}: {sentences[i]}' for i in I[0]]

['4751: Two dogs run through the brush.',
 '3846: Two dogs are running on the grass in front of the trees.',
 '2220: two dogs running in grass',
 '2649: A dog is running along the grass.']

#homework: implement all the versions mentioned in the article: Comprehensive Guide to approximate nearest neighbors algorithms|by Eyal Trabelsi|Towars Data Science

Check out the technique, document time, subjective performance
 Check to see if you search for sentence B and look in sentence A, you should get the same relatedness_score

In [49]:
index.make_direct_map()

Hugging Face Spaces, check out different models and how it is used...all the way at the bottom is the api